In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry
import datetime
import time
from collections import Counter
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from intervals import Interval
import warnings
warnings.filterwarnings('ignore')

## 垃圾数据处理

In [2]:
# 数据预处理
data=pd.read_csv('./data/Hyattsville_OpenLitterMap.csv')

In [3]:
df = data.fillna(0)# 填充缺失值

In [4]:
df

,id,verification,phone,date_taken,date_uploaded,lat,lon,picked up,address,total_litter,...,ear_plugs,elec_small,elec_large,batteries,balloons.1,life_buoy,other,custom_tag_1,custom_tag_2,custom_tag_3
0,115640,2,iPhone 11 Pro,2021-01-19 13:24:21,2021-01-19 22:15:06,38.950817,-76.959733,No,"3621, Farragut St, Hyattsville, Prince George'...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
1,115641,2,iPhone 11 Pro,2021-01-19 13:24:13,2021-01-19 22:15:10,38.950870,-76.959717,No,"3621, Farragut St, Hyattsville, Prince George'...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
2,115642,2,iPhone 11 Pro,2021-01-19 13:23:22,2021-01-19 22:15:14,38.950817,-76.959755,No,"3619, Farragut St, Hyattsville, Prince George'...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
3,115643,2,iPhone 11 Pro,2021-01-19 13:23:16,2021-01-19 22:15:18,38.950817,-76.959763,No,"3619, Farragut St, Hyattsville, Prince George'...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
4,115644,2,iPhone 11 Pro,2021-01-19 13:22:38,2021-01-19 22:15:22,38.950858,-76.959878,No,"3617, Farragut St, Hyattsville, Prince George'...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9576,269604,2,iPhone 11 Pro,2022-03-11 16:06:33,2022-03-12 00:53:08,38.954769,-76.967614,Yes,"2801, Hamilton Street, Hyattsville, Prince Geo...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
9577,269605,2,iPhone 11 Pro,2022-03-11 16:06:27,2022-03-12 00:53:13,38.954761,-76.967606,Yes,"2801, Hamilton Street, Hyattsville, Prince Geo...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
9578,269606,2,iPhone 11 Pro,2022-03-11 16:05:44,2022-03-12 00:53:15,38.954747,-76.967653,Yes,"2801, Hamilton Street, Hyattsville, Prince Geo...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
9579,269607,2,iPhone 11 Pro,2022-03-11 16:04:50,2022-03-12 00:53:18,38.954728,-76.967675,Yes,"2801, Hamilton Street, Hyattsville, Prince Geo...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0


In [5]:
sum(df['total_litter'])

21410.0

### 垃圾分类

In [5]:
# 垃圾分类
def sumc(x):# 定义合并类的函数
    count_sum = 0
    for k in x.iteritems():
        count_sum += k[1]
    return count_sum

In [6]:
df['SMOKING']=df.loc[:,'butts':'smokingOther'].apply(sumc,axis=1)
df['SMOKING'].sum()

909.0

In [7]:
df=df.drop(df.loc[:,'butts':'smokingOther'],axis=1)

In [8]:
df['FOOD']=df.loc[:,'sweetWrappers':'chewing_gum'].apply(sumc,axis=1)
df['FOOD'].sum()

2054.0

In [9]:
df=df.drop(df.loc[:,'sweetWrappers':'chewing_gum'],axis=1)

In [10]:
df['COFFEE']=df.loc[:,'coffeeCups':'coffeeOther'].apply(sumc,axis=1)
df['COFFEE'].sum()

120.0

In [11]:
df=df.drop(df.loc[:,'coffeeCups':'coffeeOther'],axis=1)

In [12]:
df['ALCOHOL']=df.loc[:,'beerCan':'alcoholOther'].apply(sumc,axis=1)
df['ALCOHOL'].sum()

5281.0

In [13]:
df=df.drop(df.loc[:,'beerCan':'alcoholOther'],axis=1)

In [14]:
df['SOFTDRINKS']=df.loc[:,'waterBottle':'softDrinkOther'].apply(sumc,axis=1)
df['SOFTDRINKS'].sum()

4659.0

In [15]:
df=df.drop(df.loc[:,'waterBottle':'softDrinkOther'],axis=1)

In [16]:
df['SANITARY']=df.loc[:,'condoms':'hand_sanitiser'].apply(sumc,axis=1)
df['SANITARY'].sum()

448.0

In [17]:
df=df.drop(df.loc[:,'condoms':'hand_sanitiser'],axis=1)

In [18]:
df['COASTAL']=df.loc[:,'microplastics':'styro_large'].apply(sumc,axis=1)
df['COASTAL'].sum()

337.0

In [19]:
df=df.drop(df.loc[:,'microplastics':'styro_large'],axis=1)

In [20]:
df['DUMPING']=df.loc[:,'small':'large'].apply(sumc,axis=1)
df['DUMPING'].sum()

3.0

In [21]:
df=df.drop(df.loc[:,'small':'large'],axis=1)

In [22]:
df['INDUSTRIAL']=df.loc[:,'oil':'industrial_other'].apply(sumc,axis=1)
df['INDUSTRIAL'].sum()

31.0

In [23]:
df['BRANDS']=df.loc[:,'aadrink':'wrigleys'].apply(sumc,axis=1)
df['BRANDS'].sum()

4542.0

In [24]:
df=df.drop(df.loc[:,'aadrink':'wrigleys'],axis=1)

In [25]:
df=df.drop(df.loc[:,'oil':'industrial_other'],axis=1)

In [26]:
df['DOGSHIT']=df.loc[:,'poo':'poo_in_bag'].apply(sumc,axis=1)
df['DOGSHIT'].sum()

21.0

In [27]:
df=df.drop(df.loc[:,'poo':'poo_in_bag'],axis=1)

In [28]:
df['ART']=df.loc[:,'item']

In [29]:
df['ART'].sum()

0.0

In [30]:
df['MATERIAL']=df.loc[:,'aluminium':'wood'].apply(sumc,axis=1)

In [31]:
df['MATERIAL'].sum()

0.0

In [32]:
df=df.drop(df.loc[:,'aluminium':'wood'],axis=1)

In [33]:
df=df.drop('item',axis=1)

In [34]:
df['OTHER']=df.loc[:,'random_litter':'other'].apply(sumc,axis=1)
df['OTHER'].sum()

3543.0

In [35]:
df=df.drop(df.loc[:,'random_litter':'other'],axis=1)

In [36]:
df.columns

Index(['id', 'verification', 'phone', 'date_taken', 'date_uploaded', 'lat',
       'lon', 'picked up', 'address', 'total_litter', 'SMOKING', 'FOOD',
       'COFFEE', 'ALCOHOL', 'SOFTDRINKS', 'SANITARY', 'COASTAL', 'DUMPING',
       'INDUSTRIAL', 'BRANDS', 'DOGSHIT', 'ART', 'MATERIAL', 'OTHER',
       'custom_tag_1', 'custom_tag_2', 'custom_tag_3'],
      dtype='object')

In [37]:
df=df.drop(df.loc[:,'custom_tag_1':'custom_tag_3'],axis=1)# 删除不需要的标签

In [38]:
df['OTHER']=df['OTHER']+df['ART']+df['DOGSHIT']+df['INDUSTRIAL']+df['DUMPING']+df['MATERIAL']

In [39]:
df['OTHER'].sum()

3598.0

In [40]:
df=df.drop(['ART','DOGSHIT','INDUSTRIAL','DUMPING','MATERIAL'],axis=1)

In [41]:
df # 分类完毕

,id,verification,phone,date_taken,date_uploaded,lat,lon,picked up,address,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER
0,115640,2,iPhone 11 Pro,2021-01-19 13:24:21,2021-01-19 22:15:06,38.950817,-76.959733,No,"3621, Farragut St, Hyattsville, Prince George'...",1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,115641,2,iPhone 11 Pro,2021-01-19 13:24:13,2021-01-19 22:15:10,38.950870,-76.959717,No,"3621, Farragut St, Hyattsville, Prince George'...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,115642,2,iPhone 11 Pro,2021-01-19 13:23:22,2021-01-19 22:15:14,38.950817,-76.959755,No,"3619, Farragut St, Hyattsville, Prince George'...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,115643,2,iPhone 11 Pro,2021-01-19 13:23:16,2021-01-19 22:15:18,38.950817,-76.959763,No,"3619, Farragut St, Hyattsville, Prince George'...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,115644,2,iPhone 11 Pro,2021-01-19 13:22:38,2021-01-19 22:15:22,38.950858,-76.959878,No,"3617, Farragut St, Hyattsville, Prince George'...",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9576,269604,2,iPhone 11 Pro,2022-03-11 16:06:33,2022-03-12 00:53:08,38.954769,-76.967614,Yes,"2801, Hamilton Street, Hyattsville, Prince Geo...",1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
9577,269605,2,iPhone 11 Pro,2022-03-11 16:06:27,2022-03-12 00:53:13,38.954761,-76.967606,Yes,"2801, Hamilton Street, Hyattsville, Prince Geo...",1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
9578,269606,2,iPhone 11 Pro,2022-03-11 16:05:44,2022-03-12 00:53:15,38.954747,-76.967653,Yes,"2801, Hamilton Street, Hyattsville, Prince Geo...",1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
9579,269607,2,iPhone 11 Pro,2022-03-11 16:04:50,2022-03-12 00:53:18,38.954728,-76.967675,Yes,"2801, Hamilton Street, Hyattsville, Prince Geo...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [42]:
# df=df[-df['date_taken'].str.contains('2019')].reset_index(drop=True) # 删除2019年的零散数据

In [43]:
# df=df[-df['date_taken'].str.contains('2020')].reset_index(drop=True) # 删除2020年的零散数据

In [44]:
df=df[-df['date_taken'].str.contains('2022')].reset_index(drop=True) # 删除2022年的零散数据

In [45]:
# df=df[-df['date_taken'].str.contains('2023')].reset_index(drop=True) # 删除2023年的零散数据

In [46]:
df=df.drop(columns=['date_uploaded'])

In [47]:
df=df.rename(columns={'date_taken':'datetime'})

In [48]:
df

,id,verification,phone,datetime,lat,lon,picked up,address,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER
0,115640,2,iPhone 11 Pro,2021-01-19 13:24:21,38.950817,-76.959733,No,"3621, Farragut St, Hyattsville, Prince George'...",1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,115641,2,iPhone 11 Pro,2021-01-19 13:24:13,38.950870,-76.959717,No,"3621, Farragut St, Hyattsville, Prince George'...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,115642,2,iPhone 11 Pro,2021-01-19 13:23:22,38.950817,-76.959755,No,"3619, Farragut St, Hyattsville, Prince George'...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,115643,2,iPhone 11 Pro,2021-01-19 13:23:16,38.950817,-76.959763,No,"3619, Farragut St, Hyattsville, Prince George'...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,115644,2,iPhone 11 Pro,2021-01-19 13:22:38,38.950858,-76.959878,No,"3617, Farragut St, Hyattsville, Prince George'...",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9523,236827,2,iPhone 11 Pro,2021-12-22 20:24:47,38.954355,-76.968170,Yes,"5400, Jamestown Road, Hyattsville, Prince Geor...",6.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,1.0
9524,237650,2,iPhone 11 Pro,2021-12-26 22:20:50,38.981400,-76.963105,Yes,"3324, Stanford Street, Hyattsville, Prince Geo...",5.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,3.0,0.0
9525,237651,2,iPhone 11 Pro,2021-12-26 22:17:19,38.981280,-76.962617,Yes,"3324, Stanford Street, Hyattsville, Prince Geo...",31.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,30.0,1.0
9526,237652,2,iPhone 11 Pro,2021-12-26 22:11:04,38.981278,-76.962687,Yes,"3324, Stanford Street, Hyattsville, Prince Geo...",108.0,20.0,7.0,0.0,44.0,36.0,0.0,0.0,47.0,1.0


In [49]:
df.to_csv('./outdata/Hyattsville_preliminary.csv')

### 合并时间

In [48]:
# df=df.rename(columns={'date_taken':'datetime'})

In [50]:
df['datetime']=pd.to_datetime(df['datetime'])

In [51]:
df['datetime']

0      2021-01-19 13:24:21
1      2021-01-19 13:24:13
2      2021-01-19 13:23:22
3      2021-01-19 13:23:16
4      2021-01-19 13:22:38
               ...        
9523   2021-12-22 20:24:47
9524   2021-12-26 22:20:50
9525   2021-12-26 22:17:19
9526   2021-12-26 22:11:04
9527   2021-12-26 19:41:51
Name: datetime, Length: 9528, dtype: datetime64[ns]

In [52]:
Date=df['datetime'].dt.date
df_dateconcat=df.loc[:,'total_litter':'OTHER'].groupby(Date).sum() # 合并同日期的垃圾数量
df_dateconcat

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER
datetime,,,,,,,,,,
2021-01-19,156.0,1.0,51.0,1.0,27.0,49.0,3.0,1.0,1.0,25.0
2021-01-20,105.0,4.0,37.0,3.0,12.0,37.0,1.0,1.0,0.0,10.0
2021-01-21,117.0,0.0,33.0,4.0,9.0,57.0,1.0,0.0,0.0,14.0
2021-01-22,183.0,1.0,95.0,0.0,19.0,57.0,1.0,1.0,3.0,6.0
2021-01-30,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2021-12-13,44.0,2.0,10.0,2.0,1.0,8.0,0.0,9.0,13.0,12.0
2021-12-14,156.0,0.0,23.0,0.0,76.0,49.0,1.0,0.0,68.0,7.0
2021-12-20,164.0,32.0,12.0,0.0,58.0,38.0,0.0,0.0,53.0,24.0


In [53]:
df_dateconcat.max()

total_litter    512.0
SMOKING          77.0
FOOD             95.0
COFFEE            7.0
ALCOHOL         226.0
SOFTDRINKS      125.0
SANITARY         19.0
COASTAL          39.0
BRANDS          230.0
OTHER           155.0
dtype: float64

### 对缺失的日期补0（舍弃）

In [53]:
start = datetime.datetime(2022, 1, 1)
end = datetime.datetime(2022, 12, 31)
date_range = pd.date_range(start, end)

In [54]:
date_range

DatetimeIndex(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
               '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08',
               '2022-01-09', '2022-01-10',
               ...
               '2022-12-22', '2022-12-23', '2022-12-24', '2022-12-25',
               '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29',
               '2022-12-30', '2022-12-31'],
              dtype='datetime64[ns]', length=365, freq='D')

In [55]:
# 筛选缺失的日期
df_date=pd.to_datetime(df_dateconcat.index) 
dateflag=date_range.isin(df_date) 

In [56]:
# date_all=np.concatenate((pd.Series(df_dateconcat.index).values,pd.Series(date_range[~dateflag].date).values),axis=0)# 合并两个时间
# date_all.size
# date_all与date_range相同，弃用data_all

In [57]:
Counter(dateflag)

Counter({True: 249, False: 116})

In [58]:
# 缺失的数据
dfdif=pd.DataFrame(np.zeros(116*10).reshape(116,10),
                   index=[pd.Series(date_range[~dateflag].date).values],
                   columns=['total_litter','SMOKING','FOOD','COFFEE','ALCOHOL','SOFTDRINKS','SANITARY','BRANDS','MATERIAL','OTHER'])

In [59]:
df_dateall=pd.concat([df_dateconcat,dfdif])# 合并

In [60]:
df_dateall=df_dateall.loc[:,'total_litter':'OTHER'].groupby(date_range).sum()

In [138]:
# df_dateall['datetime']=df_dateall.index

In [146]:
df_dateall

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,BRANDS,MATERIAL,OTHER,datetime
2022-01-01,57.0,1.0,0.0,2.0,12.0,32.0,4.0,28.0,0.0,6.0,2022-01-01
2022-01-02,48.0,1.0,2.0,0.0,7.0,17.0,10.0,20.0,0.0,11.0,2022-01-02
2022-01-03,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2022-01-03
2022-01-04,41.0,0.0,0.0,0.0,1.0,36.0,3.0,22.0,0.0,1.0,2022-01-04
2022-01-05,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2022-01-05
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27
2022-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28
2022-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29
2022-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30


In [145]:
# df_dateall.to_csv('./outdata/Krommenie_alldaylitter.csv')

In [62]:
df_dateall.max()

total_litter    466.0
SMOKING         290.0
FOOD            102.0
COFFEE           12.0
ALCOHOL          67.0
SOFTDRINKS       86.0
SANITARY         34.0
BRANDS          156.0
MATERIAL         19.0
OTHER           193.0
dtype: float64

## 天气时间数据处理

In [53]:
weather = pd.read_csv('./data/Hyattsville_weather/Hyattsville_weather.csv')

In [54]:
weather['date']=pd.to_datetime(weather['date'])

In [55]:
weather['weather'].value_counts()

Cloudy     232
Sunny       97
Showers     28
Snow         8
Name: weather, dtype: int64

In [56]:
wtcdt=pd.read_csv('./data/Hyattsville_weather/Hyattsville.csv')

In [57]:
wtcdt['date']=pd.to_datetime(wtcdt['date'])

In [58]:
wtcdt = pd.DataFrame(wtcdt).set_index('date') # 将时间作为索引

In [59]:
wtcdt

,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed
date,,,,,,,,
2021-01-01,41,36,36,25,96,60,17,3
2021-01-02,57,37,44,35,100,57,14,0
2021-01-03,43,38,39,35,93,74,13,0
2021-01-04,46,35,36,31,93,68,10,0
2021-01-05,43,39,37,33,89,70,15,0
...,...,...,...,...,...,...,...,...
2021-12-27,43,39,34,28,82,58,12,0
2021-12-28,57,40,39,24,86,29,12,0
2021-12-29,54,48,46,32,90,48,13,0


### 将天气状况加入补全后数据（舍弃）

In [ ]:
M_test=pd.concat([df_dateall,wtcdt],axis=1)# 横向合并

In [71]:
M_test

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,BRANDS,MATERIAL,OTHER,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed
2022-01-01,57.0,1.0,0.0,2.0,12.0,32.0,4.0,28.0,0.0,6.0,54,52,50,50,88,88,12,12
2022-01-02,48.0,1.0,2.0,0.0,7.0,17.0,10.0,20.0,0.0,11.0,55,50,50,45,94,77,30,12
2022-01-03,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,52,46,46,45,93,76,25,10
2022-01-04,41.0,0.0,0.0,0.0,1.0,36.0,3.0,22.0,0.0,1.0,48,37,46,32,93,70,16,5
2022-01-05,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,46,37,41,30,93,70,30,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46,41,43,36,93,71,22,9
2022-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52,43,50,41,100,87,30,20
2022-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52,45,48,41,93,76,29,15
2022-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54,39,52,39,100,81,29,12


### 将天气状况加入未补全数据

In [60]:
# 匹配时间
df_dateconcat.index

Index([2021-01-19, 2021-01-20, 2021-01-21, 2021-01-22, 2021-01-30, 2021-02-05,
       2021-02-06, 2021-02-09, 2021-02-10, 2021-02-21,
       ...
       2021-07-30, 2021-09-25, 2021-11-28, 2021-12-10, 2021-12-12, 2021-12-13,
       2021-12-14, 2021-12-20, 2021-12-22, 2021-12-26],
      dtype='object', name='datetime', length=134)

In [61]:
isindex1=wtcdt.index.isin(df_dateconcat.index)

In [62]:
wtcdt[isindex1]

,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed
date,,,,,,,,
2021-01-19,52,34,30,12,79,22,16,0
2021-01-20,45,31,29,14,76,34,28,0
2021-01-21,55,27,31,19,78,30,14,0
2021-01-22,50,33,31,17,85,29,20,0
2021-01-30,39,24,18,9,56,33,14,0
...,...,...,...,...,...,...,...,...
2021-12-13,51,33,34,22,75,35,10,0
2021-12-14,60,35,32,26,79,28,8,0
2021-12-20,41,28,25,13,70,32,10,0


In [63]:
isindex2=weather['date'].isin(df_dateconcat.index)

In [64]:
isindex2.value_counts()

False    231
True     134
Name: date, dtype: int64

In [65]:
wtdf = pd.DataFrame(weather[isindex2]).set_index('date')

In [66]:
M=pd.concat([df_dateconcat,wtdf,wtcdt[isindex1]],axis=1)# 横向合并

In [67]:
M

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER,weather,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed
2021-01-19,156.0,1.0,51.0,1.0,27.0,49.0,3.0,1.0,1.0,25.0,Cloudy,52,34,30,12,79,22,16,0
2021-01-20,105.0,4.0,37.0,3.0,12.0,37.0,1.0,1.0,0.0,10.0,Cloudy,45,31,29,14,76,34,28,0
2021-01-21,117.0,0.0,33.0,4.0,9.0,57.0,1.0,0.0,0.0,14.0,Sunny,55,27,31,19,78,30,14,0
2021-01-22,183.0,1.0,95.0,0.0,19.0,57.0,1.0,1.0,3.0,6.0,Sunny,50,33,31,17,85,29,20,0
2021-01-30,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sunny,39,24,18,9,56,33,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-13,44.0,2.0,10.0,2.0,1.0,8.0,0.0,9.0,13.0,12.0,Sunny,51,33,34,22,75,35,10,0
2021-12-14,156.0,0.0,23.0,0.0,76.0,49.0,1.0,0.0,68.0,7.0,Sunny,60,35,32,26,79,28,8,0
2021-12-20,164.0,32.0,12.0,0.0,58.0,38.0,0.0,0.0,53.0,24.0,Sunny,41,28,25,13,70,32,10,0
2021-12-22,184.0,58.0,11.0,0.0,59.0,46.0,2.0,0.0,85.0,8.0,Cloudy,51,34,36,15,93,31,23,0


In [67]:
# M_train=M.drop(M.loc[:,'SMOKING':'OTHER'],axis=1)# 舍弃分类

### 对天气描述编码

In [68]:
weather

,date,weather
0,2021-01-01,Showers
1,2021-01-02,Cloudy
2,2021-01-03,Showers
3,2021-01-04,Cloudy
4,2021-01-05,Cloudy
...,...,...
360,2021-12-27,Cloudy
361,2021-12-28,Cloudy
362,2021-12-29,Cloudy
363,2021-12-30,Showers


In [69]:
weather = pd.DataFrame(weather).set_index('date') # 将时间作为索引

In [69]:
wtdf

,weather
date,
2021-01-19,Cloudy
2021-01-20,Cloudy
2021-01-21,Sunny
2021-01-22,Sunny
2021-01-30,Sunny
...,...
2021-12-13,Sunny
2021-12-14,Sunny
2021-12-20,Sunny


In [70]:
values_wt=wtdf['weather']

In [71]:
# 整数编码
label_encoder_wt = LabelEncoder()
integer_encoded_wt = label_encoder_wt.fit_transform(values_wt)
integer_encoded_wt

array([0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 0,
       0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1, 0, 2,
       2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 0, 2, 0, 0, 0, 2, 2, 2, 2,
       0, 2])

In [72]:
# 二进制编码
wt_encoded = OneHotEncoder(sparse=False)
integer_encoded_wt = integer_encoded_wt.reshape(len(integer_encoded_wt), 1)
wt_encoded = wt_encoded.fit_transform(integer_encoded_wt)
wt_encoded.shape

(134, 3)

In [73]:
# 第一个例子的反转
inverted_wt = label_encoder_wt.inverse_transform([argmax(wt_encoded[0, :])])
inverted_wt

array(['Cloudy'], dtype=object)

### 天气特征归一化(舍弃)

In [78]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [79]:
M_train.loc[:,'Actual Mean Temperature':'Actual Sea Level Pressure'].values.reshape(-1, 1)

array([[36.59],
       [34.  ],
       [39.  ],
       ...,
       [36.  ],
       [27.  ],
       [30.58]])

In [80]:
mm = MinMaxScaler()
mm_data = mm.fit_transform(M_train.loc[:,'Actual Mean Temperature':'Actual Sea Level Pressure'].values.reshape(-1, 1))

In [81]:
mm_weather=mm_data.reshape(228,10)
mm_weather

array([[0.43559524, 0.4047619 , 0.46428571, ..., 0.42857143, 0.38095238,
        0.35809524],
       [0.44690476, 0.4047619 , 0.48809524, ..., 0.44047619, 0.38095238,
        0.35666667],
       [0.42809524, 0.35714286, 0.48809524, ..., 0.42857143, 0.35714286,
        0.35869048],
       ...,
       [0.37857143, 0.32142857, 0.51190476, ..., 0.38095238, 0.32142857,
        0.36154762],
       [0.47130952, 0.42857143, 0.51190476, ..., 0.44047619, 0.33333333,
        0.36333333],
       [0.49380952, 0.42857143, 0.57142857, ..., 0.42857143, 0.32142857,
        0.36404762]])

In [232]:
# 合并归一化后的数据
mm_weather_df=pd.DataFrame(mm_weather)
mm_weather_df=mm_weather_df.set_index(wtdf.index)
wt_encoded_df=pd.DataFrame(wt_encoded)
wt_encoded_df=wt_encoded_df.set_index(wtdf.index)
ML=pd.concat([wt_encoded_df,mm_weather_df,M_train['total_litter']],axis=1)

In [233]:
ML

,0,1,2,3,0,1,2,3,4,5,6,7,8,9,total_litter
2021-01-06,0.0,0.0,0.0,1.0,0.435595,0.404762,0.464286,0.369048,0.476190,0.178571,0.388214,0.428571,0.380952,0.358095,3.0
2021-01-07,0.0,0.0,1.0,0.0,0.446905,0.404762,0.488095,0.369048,0.476190,0.190476,0.420595,0.440476,0.380952,0.356667,2.0
2021-01-08,0.0,0.0,1.0,0.0,0.428095,0.357143,0.488095,0.369048,0.476190,0.083333,0.405833,0.428571,0.357143,0.358690,114.0
2021-01-09,0.0,1.0,0.0,0.0,0.386071,0.297619,0.511905,0.369048,0.476190,0.095238,0.352024,0.440476,0.297619,0.360476,58.0
2021-01-10,1.0,0.0,0.0,0.0,0.431429,0.333333,0.488095,0.369048,0.476190,0.142857,0.393690,0.440476,0.321429,0.360476,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-19,0.0,0.0,1.0,0.0,0.536548,0.511905,0.547619,0.392857,0.500000,0.142857,0.518690,0.535714,0.488095,0.364762,92.0
2021-12-20,1.0,0.0,0.0,0.0,0.495595,0.380952,0.535714,0.392857,0.500000,0.083333,0.418690,0.464286,0.357143,0.361905,232.0
2021-12-21,0.0,0.0,0.0,1.0,0.378571,0.321429,0.511905,0.392857,0.500000,0.059524,0.339762,0.380952,0.321429,0.361548,14.0
2022-01-20,1.0,0.0,0.0,0.0,0.471310,0.428571,0.511905,0.369048,0.476190,0.357143,0.392024,0.440476,0.333333,0.363333,5.0


In [84]:
ML_array=np.array(ML)

In [85]:
np.save('Hoorn_array', ML)

## poi分类统计（舍弃）

In [6]:
points=gpd.read_file('./data/Hyattsville_shape/points.shp')

In [7]:
points

,osm_id,timestamp,name,type,geometry
0,49159709,None,None,mini_roundabout,POINT (-76.98962 38.98505)
1,49173176,None,None,traffic_signals,POINT (-76.99501 38.98734)
2,49173198,None,None,traffic_signals,POINT (-76.99260 38.99394)
3,49175672,None,None,mini_roundabout,POINT (-76.99084 38.98727)
4,49197011,None,None,traffic_signals,POINT (-76.98781 38.98894)
...,...,...,...,...,...
5620,1545738384,None,None,street_lamp,POINT (-76.92892 38.97844)
5621,1545738385,None,None,street_lamp,POINT (-76.92891 38.97847)
5622,1582709459,None,Möge Tee,cafe,POINT (-76.93408 38.99112)
5623,1669775114,None,The Hotel at The University of Maryland,hotel,POINT (-76.93518 38.98697)


In [117]:
public=points['type'].isin(['post_box', 'pub',  'school', 'social_facility','charging_station','childcare','arts_centre',

'community_centre','kindergarten','library',  'elevator','fire_station','post_depot', 

 'social_centre', 'marketplace','police', 'theatre','yard','music_school','townhall'])

In [118]:
poi_public=points['type'][public].value_counts()

In [119]:
poi_public=poi_public.values.sum()

In [120]:
poi_public

53

In [121]:
points['type'][public]='public'

In [122]:
health=points['type'].isin(['dentist','doctors','pharmacy', 'veterinary','hospital', 'clinic'])

In [123]:
poi_health=points['type'][health].value_counts()

In [124]:
poi_health=poi_health.values.sum()

In [125]:
points['type'][health]='health'

In [126]:
leisure=points['type'].isin(['cafe', 'cinema','casino'])

In [127]:
poi_leisure=points['type'][leisure].value_counts()

In [128]:
poi_leisure=poi_leisure.values.sum()

In [129]:
poi_leisure

8

In [130]:
points['type'][leisure]='leisure'

In [131]:
catering=points['type'].isin([ 'restaurant','fast_food'])

In [132]:
poi_catering=points['type'][catering].value_counts()

In [133]:
poi_catering=poi_catering.values.sum()

In [134]:
poi_catering

28

In [135]:
points['type'][catering]='catering'

In [136]:
accommodation=points['type'].isin(['hotel','shelter','caravan_site'])

In [137]:
poi_accommodation=points['type'][accommodation].value_counts()

In [138]:
poi_accommodation=poi_accommodation.values.sum()

In [139]:
poi_accommodation

0

In [140]:
points['type'][accommodation]='accommodation'

In [141]:
shopping=points['type'].isin(['car_wash','vending_machine', 'car_sharing','ice_cream','parking', 'shower','bicycle_repair_s', 'car_rental','taxi'])

In [142]:
poi_shopping=points['type'][shopping].value_counts()

In [143]:
poi_shopping=poi_shopping.values.sum()

In [144]:
poi_shopping

6

In [145]:
points['type'][shopping]='shopping'

In [146]:
money=points['type'].isin(['atm','bank'])

In [147]:
poi_money=points['type'][money].value_counts()

In [148]:
poi_money=poi_money.values.sum()

In [149]:
poi_money

4

In [150]:
points['type'][money]='money'

In [151]:
tourism=points['type'].isin(['artwork', 'memorial', 'gallery','museum','attraction', 'cannon',  'archaeological_s',  'viewpoint', 'picnic_site'])

In [152]:
poi_tourism=points['type'][tourism].value_counts()

In [153]:
poi_tourism=poi_tourism.values.sum()

In [154]:
poi_tourism

7

In [155]:
points['type'][tourism]='tourism'

In [156]:
miscpoi=points['type'].isin(['bench','waste_basket','tower','toilets','surveillance','works','drinking_water', 'windmill','fountain', 'water_well'])

In [157]:
poi_miscpoi=points['type'][miscpoi].value_counts()

In [158]:
poi_miscpoi=poi_miscpoi.values.sum()

In [159]:
poi_miscpoi

34

In [160]:
points['type'][miscpoi]='miscpoi'

In [161]:
other=points['type'].isin(['construction','give_way','clock','street_cabinet', 'studio', 'place_of_worship',  'crane',  'communications_t','flagpole', 'chimney', 'lighthouse','tram_stop', 'yes'])

In [162]:
poi_other=points['type'][other].value_counts()

In [163]:
poi_other=poi_other.values.sum()

In [164]:
poi_other

4

In [165]:
points['type'][other]='other'

In [166]:
traffic=points['type'].isin(['mast','bicycle_parking','crossing', 'recycling', 'traffic_signals', 'signal','switch', 'level_crossing','motorway_junctio','ferry_terminal','turning_circle', 'buffer_stop', 'street_lamp','fuel', 'parking_entrance', 'stop', 'station','milestone','mini_roundabout',  'railway_crossing','junction'])

In [167]:
poi_traffic=points['type'][traffic].value_counts()

In [168]:
poi_traffic=poi_traffic.values.sum()

In [169]:
poi_traffic

207

In [170]:
points['type'][traffic]='traffic'

In [182]:
poi=pd.DataFrame(np.array([poi_public,poi_health,poi_leisure,poi_catering,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other]).reshape(1,10),
                 columns=['poi_public','poi_health','poi_leisure','poi_catering','poi_shopping','poi_money','poi_tourism','poi_miscpoi','poi_traffic','poi_other'],
                 index=['counter']
                )# 去除数量为0的poi

In [183]:
poi

,poi_public,poi_health,poi_leisure,poi_catering,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other
counter,53,17,8,28,6,4,7,34,207,4


In [173]:
# poi_other=poi_other+poi_money+poi_shopping+poi_accommodation+poi_health+poi_leisure

In [144]:
# poi['poi_other']=poi_other

In [145]:
# poi=poi.drop(['poi_health','poi_leisure','poi_accommodation','poi_shopping','poi_money'],axis=1)

In [146]:
# poi

In [184]:
poi_array=np.array(poi)

In [185]:
poi_array=np.tile(poi,(249,1))

In [186]:
poi_array

array([[ 53,  17,   8, ...,  34, 207,   4],
       [ 53,  17,   8, ...,  34, 207,   4],
       [ 53,  17,   8, ...,  34, 207,   4],
       ...,
       [ 53,  17,   8, ...,  34, 207,   4],
       [ 53,  17,   8, ...,  34, 207,   4],
       [ 53,  17,   8, ...,  34, 207,   4]], dtype=int64)

In [187]:
poi_df=pd.DataFrame(poi_array,columns=poi.columns,index=M_train.index)

In [188]:
poi_df

,poi_public,poi_health,poi_leisure,poi_catering,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other
2022-01-01,53,17,8,28,6,4,7,34,207,4
2022-01-02,53,17,8,28,6,4,7,34,207,4
2022-01-07,53,17,8,28,6,4,7,34,207,4
2022-01-10,53,17,8,28,6,4,7,34,207,4
2022-01-11,53,17,8,28,6,4,7,34,207,4
...,...,...,...,...,...,...,...,...,...,...
2022-12-25,53,17,8,28,6,4,7,34,207,4
2022-12-27,53,17,8,28,6,4,7,34,207,4
2022-12-29,53,17,8,28,6,4,7,34,207,4
2022-12-30,53,17,8,28,6,4,7,34,207,4


In [189]:
M_train=pd.concat([M_train,poi_df],axis=1)

In [190]:
M_train

,total_litter,weather,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed,poi_public,poi_health,poi_leisure,poi_catering,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other
2022-01-01,57.0,Showers,54,52,50,50,88,88,12,12,53,17,8,28,6,4,7,34,207,4
2022-01-02,48.0,Showers,55,50,50,45,94,77,30,12,53,17,8,28,6,4,7,34,207,4
2022-01-07,2.0,Showers,45,36,39,32,93,70,24,5,53,17,8,28,6,4,7,34,207,4
2022-01-10,41.0,Cloudy,45,32,41,30,100,81,9,2,53,17,8,28,6,4,7,34,207,4
2022-01-11,2.0,Foggy,41,32,39,32,100,76,14,6,53,17,8,28,6,4,7,34,207,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-25,229.0,Showers,48,45,48,45,100,93,12,5,53,17,8,28,6,4,7,34,207,4
2022-12-27,152.0,Cloudy,46,41,43,36,93,71,22,9,53,17,8,28,6,4,7,34,207,4
2022-12-29,160.0,Showers,52,45,48,41,93,76,29,15,53,17,8,28,6,4,7,34,207,4
2022-12-30,228.0,Showers,54,39,52,39,100,81,29,12,53,17,8,28,6,4,7,34,207,4


In [191]:
M_train.columns

Index(['total_litter', 'weather', 'Max Temperature', 'Min Temperature',
       'Max Dew Point', 'Min Dew Point', 'Max Humidity', 'Min Humidity',
       'Max Wind Speed', 'Min Wind Speed', 'poi_public', 'poi_health',
       'poi_leisure', 'poi_catering', 'poi_shopping', 'poi_money',
       'poi_tourism', 'poi_miscpoi', 'poi_traffic', 'poi_other'],
      dtype='object')

## 将数值型数据归一化

In [74]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [75]:
# M_test=M_test.drop(M_test.loc[:,'SMOKING':'OTHER'],axis=1)# 舍弃分类

In [76]:
M = M.drop(columns=['weather'])

In [77]:
M.loc[:,'SMOKING':'Min Wind Speed'].values.reshape(-1,1)

array([[ 1.],
       [51.],
       [ 1.],
       ...,
       [43.],
       [14.],
       [ 0.]])

In [78]:
mm = MinMaxScaler()
mm_data = mm.fit_transform(M.loc[:,'SMOKING':'Min Wind Speed'].values.reshape(-1,1))

In [79]:
mm_wp=mm_data.reshape(134,17)

In [80]:
Counter(mm_wp.flatten())

Counter({0.004347826086956522: 106,
         0.2217391304347826: 20,
         0.11739130434782609: 10,
         0.21304347826086956: 18,
         0.013043478260869566: 60,
         0.10869565217391304: 20,
         0.22608695652173913: 18,
         0.14782608695652175: 20,
         0.13043478260869565: 15,
         0.052173913043478265: 36,
         0.34347826086956523: 15,
         0.09565217391304348: 18,
         0.06956521739130435: 29,
         0.0: 381,
         0.017391304347826087: 48,
         0.16086956521739132: 16,
         0.043478260869565216: 42,
         0.1956521739130435: 7,
         0.13478260869565217: 15,
         0.12608695652173912: 20,
         0.06086956521739131: 32,
         0.33043478260869563: 20,
         0.12173913043478261: 21,
         0.14347826086956522: 25,
         0.0391304347826087: 32,
         0.24782608695652175: 17,
         0.2391304347826087: 20,
         0.08260869565217391: 19,
         0.3391304347826087: 11,
         0.41304347826086957:

In [81]:
# 合并归一化后的数据
mm_wp_df=pd.DataFrame(mm_wp)
mm_wp_df=mm_wp_df.set_index(M.index)
wt_encoded_df=pd.DataFrame(wt_encoded)
wt_encoded_df=wt_encoded_df.set_index(M.index)
ML=pd.concat([wt_encoded_df,mm_wp_df,M['total_litter']],axis=1)

In [82]:
ML

,0,1,2,0,1,2,3,4,5,6,...,8,9,10,11,12,13,14,15,16,total_litter
2021-01-19,1.0,0.0,0.0,0.004348,0.221739,0.004348,0.117391,0.213043,0.013043,0.004348,...,0.108696,0.226087,0.147826,0.130435,0.052174,0.343478,0.095652,0.069565,0.0,156.0
2021-01-20,1.0,0.0,0.0,0.017391,0.160870,0.013043,0.052174,0.160870,0.004348,0.004348,...,0.043478,0.195652,0.134783,0.126087,0.060870,0.330435,0.147826,0.121739,0.0,105.0
2021-01-21,0.0,0.0,1.0,0.000000,0.143478,0.017391,0.039130,0.247826,0.004348,0.000000,...,0.060870,0.239130,0.117391,0.134783,0.082609,0.339130,0.130435,0.060870,0.0,117.0
2021-01-22,0.0,0.0,1.0,0.004348,0.413043,0.000000,0.082609,0.247826,0.004348,0.004348,...,0.026087,0.217391,0.143478,0.134783,0.073913,0.369565,0.126087,0.086957,0.0,183.0
2021-01-30,0.0,0.0,1.0,0.000000,0.004348,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.169565,0.104348,0.078261,0.039130,0.243478,0.143478,0.060870,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-13,0.0,0.0,1.0,0.008696,0.043478,0.008696,0.004348,0.034783,0.000000,0.039130,...,0.052174,0.221739,0.143478,0.147826,0.095652,0.326087,0.152174,0.043478,0.0,44.0
2021-12-14,0.0,0.0,1.0,0.000000,0.100000,0.000000,0.330435,0.213043,0.004348,0.000000,...,0.030435,0.260870,0.152174,0.139130,0.113043,0.343478,0.121739,0.034783,0.0,156.0
2021-12-20,0.0,0.0,1.0,0.139130,0.052174,0.000000,0.252174,0.165217,0.000000,0.000000,...,0.104348,0.178261,0.121739,0.108696,0.056522,0.304348,0.139130,0.043478,0.0,164.0
2021-12-22,1.0,0.0,0.0,0.252174,0.047826,0.000000,0.256522,0.200000,0.008696,0.000000,...,0.034783,0.221739,0.147826,0.156522,0.065217,0.404348,0.134783,0.100000,0.0,184.0


In [84]:
# ML_array=np.array(ML)

In [87]:
# ML.to_csv('./outdata/Krommenie_train.csv')

## poi提取(舍弃）

In [164]:
poi1=pd.read_csv('./outdata/poi1.csv')

In [165]:
poi1['geometry']

0      POINT (5.0817843 52.6456291)
1      POINT (5.0961325 52.6457239)
2      POINT (5.0812344 52.6458317)
3      POINT (5.0968648 52.6460559)
4      POINT (5.0853898 52.6531698)
                   ...             
450    POINT (5.0931458 52.6533975)
451    POINT (5.0932666 52.6531166)
452    POINT (5.0932798 52.6532018)
453    POINT (5.0934684 52.6531761)
454    POINT (5.0936391 52.6532932)
Name: geometry, Length: 455, dtype: object

In [166]:
points['geometry'].astype(str)

0       POINT (5.0624231 52.6350804)
1       POINT (5.0085054 52.6350184)
2         POINT (5.0578571 52.63695)
3       POINT (5.0572411 52.6381662)
4       POINT (5.0523931 52.6417891)
                    ...             
2397     POINT (5.0472634 52.647675)
2398    POINT (5.0469239 52.6475957)
2399    POINT (5.0465883 52.6476558)
2400    POINT (5.0473351 52.6479847)
2401    POINT (5.0471709 52.6476163)
Name: geometry, Length: 2402, dtype: object

In [167]:
poi1_flag=points['geometry'].astype(str).isin(poi1['geometry'])

In [168]:
poi1_flag.value_counts()

False    1947
True      455
Name: geometry, dtype: int64

In [169]:
poi1=points[poi1_flag].reset_index(drop=True)# 第一类poi

In [170]:
poi1.to_file('./poi1.shp')# 输出为shapefile文件

In [171]:
poi1[['osm_id','type','geometry']].set_index(['osm_id'])

,type,geometry
osm_id,,
47077200,traffic,POINT (5.08178 52.64563)
47080156,traffic,POINT (5.09613 52.64572)
47081177,traffic,POINT (5.08123 52.64583)
47081536,traffic,POINT (5.09686 52.64606)
47097804,traffic,POINT (5.08539 52.65317)
...,...,...
49020516,miscpoi,POINT (5.09315 52.65340)
49020522,miscpoi,POINT (5.09327 52.65312)
49020523,miscpoi,POINT (5.09328 52.65320)


## 时间分段标签

In [83]:
df_date=pd.to_datetime(df_dateconcat.index)

In [84]:
df_date

DatetimeIndex(['2021-01-19', '2021-01-20', '2021-01-21', '2021-01-22',
               '2021-01-30', '2021-02-05', '2021-02-06', '2021-02-09',
               '2021-02-10', '2021-02-21',
               ...
               '2021-07-30', '2021-09-25', '2021-11-28', '2021-12-10',
               '2021-12-12', '2021-12-13', '2021-12-14', '2021-12-20',
               '2021-12-22', '2021-12-26'],
              dtype='datetime64[ns]', name='datetime', length=134, freq=None)

In [85]:
is_weekly=df_date.weekday==(5 or 6) # 一周结束

In [86]:
df_date.weekday==(5 or 6)

array([False, False, False, False,  True, False,  True, False, False,
       False, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False,  True, False, False,
       False, False, False,  True, False, False, False,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False,  True, False, False, False, False, False,  True,
       False, False,

In [87]:
holiday=pd.to_datetime([
        '2021-01-01','2021-01-18','2021-02-15','2021-05-31','2021-07-04','2021-09-06','2021-10-11','2021-11-11','2021-11-25','2021-12-24',
         '2021-12-25'])

In [88]:
is_holiday=df_date.isin(holiday)

In [89]:
is_holiday=np.array(is_holiday)+np.array(df_date.weekday==(5 or 6))

In [90]:
Date2=pd.DataFrame(np.zeros([134,3]),columns=['daily','weekly','holiday'])

In [91]:
Date2=Date2.set_index(df_date.date)

In [92]:
ML1=pd.concat([Date2,ML],axis=1)

In [93]:
ML1

,daily,weekly,holiday,0,1,2,0,1,2,3,...,8,9,10,11,12,13,14,15,16,total_litter
2021-01-19,0.0,0.0,0.0,1.0,0.0,0.0,0.004348,0.221739,0.004348,0.117391,...,0.108696,0.226087,0.147826,0.130435,0.052174,0.343478,0.095652,0.069565,0.0,156.0
2021-01-20,0.0,0.0,0.0,1.0,0.0,0.0,0.017391,0.160870,0.013043,0.052174,...,0.043478,0.195652,0.134783,0.126087,0.060870,0.330435,0.147826,0.121739,0.0,105.0
2021-01-21,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.143478,0.017391,0.039130,...,0.060870,0.239130,0.117391,0.134783,0.082609,0.339130,0.130435,0.060870,0.0,117.0
2021-01-22,0.0,0.0,0.0,0.0,0.0,1.0,0.004348,0.413043,0.000000,0.082609,...,0.026087,0.217391,0.143478,0.134783,0.073913,0.369565,0.126087,0.086957,0.0,183.0
2021-01-30,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.004348,0.000000,0.000000,...,0.000000,0.169565,0.104348,0.078261,0.039130,0.243478,0.143478,0.060870,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-13,0.0,0.0,0.0,0.0,0.0,1.0,0.008696,0.043478,0.008696,0.004348,...,0.052174,0.221739,0.143478,0.147826,0.095652,0.326087,0.152174,0.043478,0.0,44.0
2021-12-14,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.100000,0.000000,0.330435,...,0.030435,0.260870,0.152174,0.139130,0.113043,0.343478,0.121739,0.034783,0.0,156.0
2021-12-20,0.0,0.0,0.0,0.0,0.0,1.0,0.139130,0.052174,0.000000,0.252174,...,0.104348,0.178261,0.121739,0.108696,0.056522,0.304348,0.139130,0.043478,0.0,164.0
2021-12-22,0.0,0.0,0.0,1.0,0.0,0.0,0.252174,0.047826,0.000000,0.256522,...,0.034783,0.221739,0.147826,0.156522,0.065217,0.404348,0.134783,0.100000,0.0,184.0


In [94]:
ML1['daily']=1.0

In [95]:
ML1[is_weekly].index

Index([2021-01-30, 2021-02-06, 2021-02-27, 2021-03-13, 2021-03-27, 2021-04-17,
       2021-05-01, 2021-05-29, 2021-06-05, 2021-06-19, 2021-06-26, 2021-07-03,
       2021-07-10, 2021-07-17, 2021-07-24, 2021-09-25],
      dtype='object')

In [96]:
ML1.loc[is_weekly,'weekly']=1.0

In [97]:
ML1['weekly'].value_counts()

0.0    118
1.0     16
Name: weekly, dtype: int64

In [98]:
ML1.loc[is_holiday,'holiday']=1.0

In [99]:
ML1['holiday'].value_counts()

0.0    116
1.0     18
Name: holiday, dtype: int64

In [100]:
ML1

,daily,weekly,holiday,0,1,2,0,1,2,3,...,8,9,10,11,12,13,14,15,16,total_litter
2021-01-19,1.0,0.0,0.0,1.0,0.0,0.0,0.004348,0.221739,0.004348,0.117391,...,0.108696,0.226087,0.147826,0.130435,0.052174,0.343478,0.095652,0.069565,0.0,156.0
2021-01-20,1.0,0.0,0.0,1.0,0.0,0.0,0.017391,0.160870,0.013043,0.052174,...,0.043478,0.195652,0.134783,0.126087,0.060870,0.330435,0.147826,0.121739,0.0,105.0
2021-01-21,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.143478,0.017391,0.039130,...,0.060870,0.239130,0.117391,0.134783,0.082609,0.339130,0.130435,0.060870,0.0,117.0
2021-01-22,1.0,0.0,0.0,0.0,0.0,1.0,0.004348,0.413043,0.000000,0.082609,...,0.026087,0.217391,0.143478,0.134783,0.073913,0.369565,0.126087,0.086957,0.0,183.0
2021-01-30,1.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.004348,0.000000,0.000000,...,0.000000,0.169565,0.104348,0.078261,0.039130,0.243478,0.143478,0.060870,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-13,1.0,0.0,0.0,0.0,0.0,1.0,0.008696,0.043478,0.008696,0.004348,...,0.052174,0.221739,0.143478,0.147826,0.095652,0.326087,0.152174,0.043478,0.0,44.0
2021-12-14,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.100000,0.000000,0.330435,...,0.030435,0.260870,0.152174,0.139130,0.113043,0.343478,0.121739,0.034783,0.0,156.0
2021-12-20,1.0,0.0,0.0,0.0,0.0,1.0,0.139130,0.052174,0.000000,0.252174,...,0.104348,0.178261,0.121739,0.108696,0.056522,0.304348,0.139130,0.043478,0.0,164.0
2021-12-22,1.0,0.0,0.0,1.0,0.0,0.0,0.252174,0.047826,0.000000,0.256522,...,0.034783,0.221739,0.147826,0.156522,0.065217,0.404348,0.134783,0.100000,0.0,184.0


In [101]:
ML1.to_csv('./outdata/Hyattsville_ML.csv')